<a href="https://colab.research.google.com/github/kiat/MET-CS777/blob/master/Colab_TPC_H_Example_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://mirror.cc.columbia.edu/pub/software/apache/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark
!rm -rf spark-3.0.0-preview2-bin-hadoop2.7.tgz

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

import findspark
findspark.init("spark-3.0.0-preview2-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession
from pyspark import SparkContext


import sys
from operator import add

from pyspark import SparkContext
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
import sys
import requests
from operator import add

from pyspark.sql.types import *
from pyspark.sql import functions as func

from pyspark.sql.functions import lit
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from pyspark.sql.functions import array

from pyspark.sql import SQLContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

# conf = SparkConf(appName="TPCH-Example")
# create spark context with the above configuration
# sc = SparkContext(conf=conf)

sqlContext = SQLContext(sc)


In [0]:
! wget -q https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/customer.tbl
! wget -q https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/lineitem.tbl
! wget -q https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/nation.tbl
! wget -q https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/orders.tbl
! wget -q https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/region.tbl
! wget -q https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/supplier.tbl
! wget -q https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/part.tbl
! wget -q https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/region.tbl
! wget -q https://github.com/kiat/MET-CS777/raw/master/TPC-H-Example/tpch_tables_scale_0.1/partsupp.tbl

In [4]:
! ls -la 


total 105344
drwxr-xr-x  1 root      root     4096 Feb 10 22:30 .
drwxr-xr-x  1 root      root     4096 Feb 10 22:25 ..
drwxr-xr-x  1 root      root     4096 Feb  7 17:12 .config
-rw-r--r--  1 root      root  2426178 Feb 10 22:30 customer.tbl
-rw-r--r--  1 root      root 74247152 Feb 10 22:30 lineitem.tbl
-rw-r--r--  1 root      root     2257 Feb 10 22:30 nation.tbl
-rw-r--r--  1 root      root 16893215 Feb 10 22:30 orders.tbl
-rw-r--r--  1 root      root 11728237 Feb 10 22:30 partsupp.tbl
-rw-r--r--  1 root      root  2391154 Feb 10 22:30 part.tbl
-rw-r--r--  1 root      root      412 Feb 10 22:30 region.tbl
-rw-r--r--  1 root      root      412 Feb 10 22:30 region.tbl.1
drwxr-xr-x  1 root      root     4096 Feb  5 18:37 sample_data
drwxr-xr-x 13 110302528 1000     4096 Dec 17 03:48 spark-3.0.0-preview2-bin-hadoop2.7
-rw-r--r--  1 root      root   139679 Feb 10 22:30 supplier.tbl


In [5]:
lineitems = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("lineitem.tbl")
lineitems.show()

+--------+-------+-------+----------+--------+-------------+--------+----+----------+----------+----------+----------+-----------+-----------------+--------+--------------------+
|ORDERKEY|PARTKEY|SUPPKEY|LINENUMBER|QUANTITY|EXTENDEDPRICE|DISCOUNT| TAX|RETURNFLAG|LINESTATUS|  SHIPDATE|COMMITDATE|RECEIPTDATE|     SHIPINSTRUCT|SHIPMODE|             COMMENT|
+--------+-------+-------+----------+--------+-------------+--------+----+----------+----------+----------+----------+-----------+-----------------+--------+--------------------+
|       1|  15519|    785|         1|      17|     24386.67|    0.04|0.02|         N|         O|1996-03-13|1996-02-12| 1996-03-22|DELIVER IN PERSON|   TRUCK|egular courts abo...|
|       1|   6731|    732|         2|      36|     58958.28|    0.09|0.06|         N|         O|1996-04-12|1996-02-28| 1996-04-20| TAKE BACK RETURN|    MAIL|ly final dependen...|
|       1|   6370|    371|         3|       8|     10210.96|     0.1|0.02|         N|         O|1996-01-2

In [0]:

customer = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("customer.tbl")
orders = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("orders.tbl")
lineitems = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("lineitem.tbl")
part = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("part.tbl")
supplier = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("supplier.tbl")
partsupp = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("partsupp.tbl")
region = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("region.tbl")
nation = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("nation.tbl")



In [7]:
nation.show()

+---------+----------+---------+--------------------+
|NATIONKEY|      NAME|REGIONKEY|             COMMENT|
+---------+----------+---------+--------------------+
|        0|   ALGERIA|        0| haggle. carefull...|
|        1| ARGENTINA|        1|al foxes promise ...|
|        2|    BRAZIL|        1|y alongside of th...|
|        3|    CANADA|        1|eas hang ironic, ...|
|        4|     EGYPT|        4|y above the caref...|
|        5|  ETHIOPIA|        0|ven packages wake...|
|        6|    FRANCE|        3|refully final req...|
|        7|   GERMANY|        3|l platelets. regu...|
|        8|     INDIA|        2|ss excuses cajole...|
|        9| INDONESIA|        2| slyly express as...|
|       10|      IRAN|        4|efully alongside ...|
|       11|      IRAQ|        4|nic deposits boos...|
|       12|     JAPAN|        2|ously. final, exp...|
|       13|    JORDAN|        4|ic deposits are b...|
|       14|     KENYA|        0| pending excuses ...|
|       15|   MOROCCO|      